![Neptune + PyTorch](https://neptune.ai/wp-content/uploads/2023/09/pytorch.svg)

# Neptune + PyTorch

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/pytorch/notebooks/Neptune_PyTorch.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/pytorch/notebooks/Neptune_PyTorch.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/common/org/pytorch-integration/runs/details?viewId=standard-view&detailsTab=dashboard&dashboardId=9920962e-ff6a-4dea-b551-88006799b116&shortId=PYTOR1-7411&type=run"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a><a target="_blank" href="https://docs.neptune.ai/integrations/pytorch/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

Introduction

This guide will show you how to:

* Create a NeptuneLogger()
* Log training metrics to Neptune using NeptuneLogger()
* Upload model checkpoints to Neptune using NeptuneLogger()
* Log model predictions to Neptune using NeptuneLogger()

## Before you start

This notebook example lets you try out Neptune anonymously, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
%pip install -U neptune neptune-pytorch numpy torch torchvision torchviz

## Start a run

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in a public project. **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
import neptune
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.


In [ ]:
import neptune

run = neptune.init_run(
    api_token=neptune.ANONYMOUS_API_TOKEN,  # replace with your own
    project="common/pytorch-integration",  # replace with your own
)

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

We'll use the `run` object we just created to log metadata. You'll see the metadata appear in the app.

### Imports

In [ ]:
import torch
from torch import nn
from torch import optim
from torchvision import transforms, datasets
import numpy as np

### Hyperparameters for training

In [ ]:
parameters = {
    "lr": 1e-2,
    "bs": 128,
    "input_sz": 32 * 32 * 3,
    "n_classes": 10,
    "model_filename": "basemodel",
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "epochs": 2,
}

### Model

In [ ]:
class Model(nn.Module):
    def __init__(self, input_sz, hidden_dim, n_classes):
        super(Model, self).__init__()
        self.seq_model = nn.Sequential(
            nn.Linear(input_sz, hidden_dim * 2),
            nn.ReLU(),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, n_classes),
        )

    def forward(self, input):
        x = input.view(-1, 32 * 32 * 3)
        return self.seq_model(x)


model = Model(parameters["input_sz"], parameters["input_sz"], parameters["n_classes"]).to(
    parameters["device"]
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=parameters["lr"])

### Download and transform the data for training

In [ ]:
data_dir = "data/CIFAR10"
compressed_ds = "./data/CIFAR10/cifar-10-python.tar.gz"
data_tfms = {
    "train": transforms.Compose(
        [
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
    "val": transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
}

trainset = datasets.CIFAR10(data_dir, transform=data_tfms["train"], download=True)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=parameters["bs"], shuffle=True, num_workers=0
)
validset = datasets.CIFAR10(data_dir, train=False, transform=data_tfms["train"], download=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=parameters["bs"], num_workers=0)

classes = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

### (Neptune) Create NeptuneLogger

In [ ]:
from neptune_pytorch import NeptuneLogger

npt_logger = NeptuneLogger(
    run, model=model, log_model_diagram=True, log_gradients=True, log_parameters=True, log_freq=30
)

### (Neptune) Log hyperparams

In [ ]:
from neptune.utils import stringify_unsupported

# (Neptune) The base_namespace attribute of the logger can be used to log metadata consistently
# under the 'base_namespace' namespace.
run[npt_logger.base_namespace]["hyperparams"] = stringify_unsupported(parameters)

### (Neptune) Log metrics while training

In [ ]:
for epoch in range(parameters["epochs"]):
    for i, (x, y) in enumerate(trainloader, 0):
        x, y = x.to(parameters["device"]), y.to(parameters["device"])
        optimizer.zero_grad()
        outputs = model(x)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, y)
        acc = (torch.sum(preds == y.data)) / len(x)

        # Log after every 30 steps
        if i % 30 == 0:
            run[npt_logger.base_namespace]["batch/loss"].append(loss.item())
            run[npt_logger.base_namespace]["batch/acc"].append(acc.item())

        loss.backward()
        optimizer.step()

    # Checkpoint number is automatically incremented on subsequent call.
    # Call 1 -> ckpt_1.pt
    # Call 2 -> ckpt_2.pt
    # npt_logger.log_checkpoint()  # uncomment to log model checkpoints to the run

### (Neptune) Log prediction from model

In [ ]:
from neptune.types import File

dataiter = iter(validloader)
images, labels = next(dataiter)

# Predict batch of n_samples
n_samples = 10
imgs = images[:n_samples].to(parameters["device"])
probs = torch.nn.functional.softmax(model(imgs), dim=1)

# Decode probs and Log tensors as image
for i, ps in enumerate(probs):
    pred = classes[torch.argmax(ps)]
    ground_truth = classes[labels[i]]
    description = f"pred: {pred} | ground truth: {ground_truth}"

    # Log Series of Tensors as Image and Predictions.
    run[npt_logger.base_namespace]["predictions"].append(
        File.as_image(imgs[i].cpu().squeeze().permute(2, 1, 0).clip(0, 1)),
        name=f"{i}_{pred}_{ground_truth}",
        description=description,
    )

### (Neptune) Save final model

In [ ]:
# Log final model as "model.pt"
# npt_logger.log_model("model")  # uncomment to log the final model to the run

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()

## Explore the results in Neptune

Go to the run link and explore metadata (metrics, params, predictions) that were logged to the run in Neptune.

You can also check out an [example run](https://app.neptune.ai/o/common/org/pytorch-integration/runs/details?viewId=standard-view&detailsTab=dashboard&dashboardId=9920962e-ff6a-4dea-b551-88006799b116&shortId=PYTOR1-7411&type=run).